# Nonparametric ML Models - Cumulative Lab

## Introduction

In this cumulative lab, you will apply two nonparametric models you have just learned — k-nearest neighbors and decision trees — to the forest cover dataset.

## Objectives

* Practice identifying and applying appropriate preprocessing steps
* Perform an iterative modeling process, starting from a baseline model
* Explore multiple model algorithms, and tune their hyperparameters
* Practice choosing a final model across multiple model algorithms and evaluating its performance

## Your Task: Complete an End-to-End ML Process with Nonparametric Models on the Forest Cover Dataset

![line of pine trees](https://curriculum-content.s3.amazonaws.com/data-science/images/trees.jpg)

Photo by <a href="https://unsplash.com/@michaelbenz?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Michael Benz</a> on <a href="/s/photos/forest?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>

### Business and Data Understanding

To repeat the previous description:

> Here we will be using an adapted version of the forest cover dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/covertype). Each record represents a 30 x 30 meter cell of land within Roosevelt National Forest in northern Colorado, which has been labeled as `Cover_Type` 1 for "Cottonwood/Willow" and `Cover_Type` 0 for "Ponderosa Pine". (The original dataset contained 7 cover types but we have simplified it.)

The task is to predict the `Cover_Type` based on the available cartographic variables:

In [1]:
# Run this cell without changes
import pandas as pd

df = pd.read_csv('data/forest_cover.csv')
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Cover_Type
0,2553,235,17,351,95,780,188,253,199,1410,...,0,0,0,0,0,0,0,0,0,0
1,2011,344,17,313,29,404,183,211,164,300,...,0,0,0,0,0,0,0,0,0,0
2,2022,24,13,391,42,509,212,212,134,421,...,0,0,0,0,0,0,0,0,0,0
3,2038,50,17,408,71,474,226,200,102,283,...,0,0,0,0,0,0,0,0,0,0
4,2018,341,27,351,34,390,152,188,168,190,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38496,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,0
38497,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,0
38498,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,0
38499,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,0


> As you can see, we have over 38,000 rows, each with 52 feature columns and 1 target column:

> * `Elevation`: Elevation in meters
> * `Aspect`: Aspect in degrees azimuth
> * `Slope`: Slope in degrees
> * `Horizontal_Distance_To_Hydrology`: Horizontal dist to nearest surface water features in meters
> * `Vertical_Distance_To_Hydrology`: Vertical dist to nearest surface water features in meters
> * `Horizontal_Distance_To_Roadways`: Horizontal dist to nearest roadway in meters
> * `Hillshade_9am`: Hillshade index at 9am, summer solstice
> * `Hillshade_Noon`: Hillshade index at noon, summer solstice
> * `Hillshade_3pm`: Hillshade index at 3pm, summer solstice
> * `Horizontal_Distance_To_Fire_Points`: Horizontal dist to nearest wildfire ignition points, meters
> * `Wilderness_Area_x`: Wilderness area designation (3 columns)
> * `Soil_Type_x`: Soil Type designation (39 columns)
> * `Cover_Type`: 1 for cottonwood/willow, 0 for ponderosa pine

This is also an imbalanced dataset, since cottonwood/willow trees are relatively rare in this forest:

In [3]:
# Run this cell without changes
print("Raw Counts")
print(df["Cover_Type"].value_counts())
print()
print("Percentages")
print(df["Cover_Type"].value_counts(normalize=True))

Raw Counts
Cover_Type
0    35754
1     2747
Name: count, dtype: int64

Percentages
Cover_Type
0    0.928651
1    0.071349
Name: proportion, dtype: float64


Thus, a baseline model that always chose the majority class would have an accuracy of over 92%. Therefore we will want to report additional metrics at the end.

### Previous Best Model

In a previous lab, we used SMOTE to create additional synthetic data, then tuned the hyperparameters of a logistic regression model to get the following final model metrics:

* **Log loss:** 0.13031294393913376
* **Accuracy:** 0.9456679825472678
* **Precision:** 0.6659919028340081
* **Recall:** 0.47889374090247455

In this lab, you will try to beat those scores using more-complex, nonparametric models.

### Modeling

Although you may be aware of some additional model algorithms available from scikit-learn, for this lab you will be focusing on two of them: k-nearest neighbors and decision trees. Here are some reminders about these models:

#### kNN - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

This algorithm — unlike linear models or tree-based models — does not emphasize learning the relationship between the features and the target. Instead, for a given test record, it finds the most similar records in the training set and returns an average of their target values.

* **Training speed:** Fast. In theory it's just saving the training data for later, although the scikit-learn implementation has some additional logic "under the hood" to make prediction faster.
* **Prediction speed:** Very slow. The model has to look at every record in the training set to find the k closest to the new record.
* **Requires scaling:** Yes. The algorithm to find the nearest records is distance-based, so it matters that distances are all on the same scale.
* **Key hyperparameters:** `n_neighbors` (how many nearest neighbors to find; too few neighbors leads to overfitting, too many leads to underfitting), `p` and `metric` (what kind of distance to use in defining "nearest" neighbors)

#### Decision Trees - [documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Similar to linear models (and unlike kNN), this algorithm emphasizes learning the relationship between the features and the target. However, unlike a linear model that tries to find linear relationships between each of the features and the target, decision trees look for ways to split the data based on features to decrease the entropy of the target in each split.

* **Training speed:** Slow. The model is considering splits based on as many as all of the available features, and it can split on the same feature multiple times. This requires exponential computational time that increases based on the number of columns as well as the number of rows.
* **Prediction speed:** Medium fast. Producing a prediction with a decision tree means applying several conditional statements, which is slower than something like logistic regression but faster than kNN.
* **Requires scaling:** No. This model is not distance-based. You also can use a `LabelEncoder` rather than `OneHotEncoder` for categorical data, since this algorithm doesn't necessarily assume that the distance between `1` and `2` is the same as the distance between `2` and `3`.
* **Key hyperparameters:** Many features relating to "pruning" the tree. By default they are set so the tree can overfit, and by setting them higher or lower (depending on the hyperparameter) you can reduce overfitting, but too much will lead to underfitting. These are: `max_depth`, `min_samples_split`, `min_samples_leaf`, `min_weight_fraction_leaf`, `max_features`, `max_leaf_nodes`, and `min_impurity_decrease`. You can also try changing the `criterion` to "entropy" or the `splitter` to "random" if you want to change the splitting logic.

### Requirements

#### 1. Prepare the Data for Modeling

#### 2. Build a Baseline kNN Model

#### 3. Build Iterative Models to Find the Best kNN Model

#### 4. Build a Baseline Decision Tree Model

#### 5. Build Iterative Models to Find the Best Decision Tree Model

#### 6. Choose and Evaluate an Overall Best Model

## 1. Prepare the Data for Modeling

The target is `Cover_Type`. In the cell below, split `df` into `X` and `y`, then perform a train-test split with `random_state=42` and `stratify=y` to create variables with the standard `X_train`, `X_test`, `y_train`, `y_test` names.

Include the relevant imports as you go.

In [5]:
# Your code here
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
X = df.drop(columns='Cover_Type')  # All columns except the target
y = df['Cover_Type']  # Target variable only

# Perform stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2,        # 20% for test set
    random_state=42,      # For reproducibility
    stratify=y           # Maintain class balance in splits
)

# Verify the shapes and class distribution
print("Shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

print("\nClass distribution in y_train:")
print(y_train.value_counts(normalize=True))

print("\nClass distribution in y_test:")
print(y_test.value_counts(normalize=True))



Shapes:
X_train: (30800, 52), y_train: (30800,)
X_test: (7701, 52), y_test: (7701,)

Class distribution in y_train:
Cover_Type
0    0.928636
1    0.071364
Name: proportion, dtype: float64

Class distribution in y_test:
Cover_Type
0    0.928711
1    0.071289
Name: proportion, dtype: float64


Now, instantiate a `StandardScaler`, fit it on `X_train`, and create new variables `X_train_scaled` and `X_test_scaled` containing values transformed with the scaler.

In [11]:
# Your code here
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Instantiate the scaler
scaler = StandardScaler()

# Fit the scaler on training data and transform both train and test sets
X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform training data
X_test_scaled = scaler.transform(X_test)       # Transform test data using same scaling

# Convert back to DataFrames (optional, but maintains column names)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

# Verify the scaling
print("X_train_scaled - First 5 rows:")
display(X_train_scaled.head())
print("\nX_test_scaled - First 5 rows:")
display(X_test_scaled.head())

# Show mean and std for verification (should be ~0 mean and ~1 std for training set)
print("\nTraining set statistics after scaling:")
print("Means:", X_train_scaled.mean().values[:5])  # First 5 features
print("Standard deviations:", X_train_scaled.std().values[:5])

X_train_scaled - First 5 rows:


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
6417,-1.495584,1.074229,1.589800,-0.989689,-0.340442,-1.217970,-2.480485,-0.139465,1.869558,-0.569849,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36343,0.623823,0.446347,0.925570,-0.367750,0.518887,1.130281,-0.790589,1.317505,1.207315,-0.703475,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1693,-0.894406,-0.490792,0.039929,-1.114077,-0.795381,0.055940,1.147822,0.079081,-1.006467,0.781687,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26095,1.296327,0.933659,-1.842057,0.046875,-1.064974,0.164039,0.128913,0.916838,0.601837,-0.155607,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29989,1.459359,0.062120,0.039929,0.060696,0.384090,1.456242,0.476833,1.171808,0.185570,3.547753,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0



X_test_scaled - First 5 rows:


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
4174,1.367654,0.099605,0.372045,-0.782376,-0.289893,1.667451,0.352576,1.171808,0.223412,3.450397,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31802,0.664581,-0.078451,0.482750,-0.989689,-0.997575,0.666285,0.725347,0.843990,-0.325303,-0.153698,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11424,-0.690617,1.711484,1.479095,2.030167,0.485187,-0.604298,-1.387023,-2.033525,-0.041485,-0.447676,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37740,1.428790,-1.081189,-0.845711,2.403331,-0.593186,-0.196847,0.625942,-0.175889,-0.476673,0.455257,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1748,-0.104723,-0.556392,0.150634,1.117991,1.748906,-0.467926,1.172673,-0.103040,-1.138915,1.466999,...,0.0,-0.05135,-0.012742,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Training set statistics after scaling:
Means: [ 3.36815712e-16 -6.78245339e-17 -1.28958893e-16 -7.23230999e-17
  2.88369617e-17]
Standard deviations: [1.00001623 1.00001623 1.00001623 1.00001623 1.00001623]


The following code checks that everything is set up correctly:

In [13]:
# Run this cell without changes

# Checking that df was separated into correct X and y
assert type(X) == pd.DataFrame and X.shape == (38501, 52)
assert type(y) == pd.Series and y.shape == (38501,)

# Checking the train-test split
assert type(X_train) == pd.DataFrame and X_train.shape == (28875, 52)
assert type(X_test) == pd.DataFrame and X_test.shape == (9626, 52)
assert type(y_train) == pd.Series and y_train.shape == (28875,)
assert type(y_test) == pd.Series and y_test.shape == (9626,)

# Checking the scaling
assert X_train_scaled.shape == X_train.shape
assert round(X_train_scaled[0][0], 3) == -0.636
assert X_test_scaled.shape == X_test.shape
assert round(X_test_scaled[0][0], 3) == -1.370

AssertionError: 

## 2. Build a Baseline kNN Model

Build a scikit-learn kNN model with default hyperparameters. Then use `cross_val_score` with `scoring="neg_log_loss"` to find the mean log loss for this model (passing in `X_train_scaled` and `y_train` to `cross_val_score`). You'll need to find the mean of the cross-validated scores, and negate the value (either put a `-` at the beginning or multiply by `-1`) so that your answer is a log loss rather than a negative log loss.

Call the resulting score `knn_baseline_log_loss`.

Your code might take a minute or more to run.

In [19]:
# Replace None with appropriate code

# Relevant imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Creating the model
knn_baseline_model = KNeighborsClassifier()
cv_scores = cross_val_score(
    knn_baseline_model,
    X_train_scaled,
    y_train,
    cv=5,  # 5-fold cross-validation
    scoring='neg_log_loss',
    n_jobs=-1  # Use all available CPU cores
)

# Perform cross-validation
knn_baseline_log_loss = -np.mean(cv_scores)

knn_baseline_log_loss

0.1176651591100023

Our best logistic regression model had a log loss of 0.13031294393913376

Is this model better? Compare it in terms of metrics and speed.

In [ ]:
# Replace None with appropriate text
"""
None
"""

## 3. Build Iterative Models to Find the Best kNN Model

Build and evaluate at least two more kNN models to find the best one. Explain why you are changing the hyperparameters you are changing as you go. These models will be *slow* to run, so be thinking about what you might try next as you run them.

In [21]:
# Your code here (add more cells as needed)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Model 1: Larger neighborhood with distance weighting
knn_model1 = KNeighborsClassifier(
    n_neighbors=15,        # Increased from default 5 to reduce noise
    weights='distance',    # Closer neighbors matter more
    p=2,                   # Euclidean distance
    n_jobs=-1              # Parallel processing
)

# Cross-validate
cv_scores1 = cross_val_score(
    knn_model1,
    X_train_scaled,
    y_train,
    cv=5,
    scoring='neg_log_loss'
)
knn_log_loss1 = -np.mean(cv_scores1)

print(f"Model 1 Log Loss: {knn_log_loss1:.5f}")
print(f"Improvement: {0.1303 - knn_log_loss1:.5f} over logistic regression")

Model 1 Log Loss: 0.06559
Improvement: 0.06471 over logistic regression


In [25]:
# Your code here (add more cells as needed)
knn_model2 = KNeighborsClassifier(
    n_neighbors=25,        # Even larger neighborhood
    weights='distance',
    p=1,                   # Manhattan distance (better for high dimensions)
    metric='manhattan',
    n_jobs=-1
)

cv_scores2 = cross_val_score(
    knn_model2,
    X_train_scaled,
    y_train,
    cv=5,
    scoring='neg_log_loss'
)
knn_log_loss2 = -np.mean(cv_scores2)

print(f"\nModel 2 Log Loss: {knn_log_loss2:.5f}")
print(f"Improvement: {0.1303 - knn_log_loss2:.5f} over logistic regression")


Model 2 Log Loss: 0.05725
Improvement: 0.07305 over logistic regression


In [28]:
# Your code here (add more cells as needed)
knn_model3 = KNeighborsClassifier(
    n_neighbors=20,        # Middle ground between previous models
    weights='distance',
    p=1,                   # Manhattan
    leaf_size=30,          # Optimizes tree structure
    metric='manhattan',
    n_jobs=-1
)

cv_scores3 = cross_val_score(
    knn_model3,
    X_train_scaled,
    y_train,
    cv=5,
    scoring='neg_log_loss'
)
knn_log_loss3 = -np.mean(cv_scores3)

print(f"\nModel 3 Log Loss: {knn_log_loss3:.5f}")
print(f"Improvement: {0.1303 - knn_log_loss3:.5f} over logistic regression")


Model 3 Log Loss: 0.05735
Improvement: 0.07295 over logistic regression


## 4. Build a Baseline Decision Tree Model

Now that you have chosen your best kNN model, start investigating decision tree models. First, build and evaluate a baseline decision tree model, using default hyperparameters (with the exception of `random_state=42` for reproducibility).

(Use cross-validated log loss, just like with the previous models.)

In [30]:
# Your code here
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Instantiate baseline Decision Tree with random_state for reproducibility
dt_baseline = DecisionTreeClassifier(random_state=42)

# Perform cross-validation with log loss scoring
dt_cv_scores = cross_val_score(
    dt_baseline,
    X_train,  # Using original unscaled data (trees don't require scaling)
    y_train,
    cv=5,
    scoring='neg_log_loss',
    n_jobs=-1  # Parallel processing
)

# Calculate mean log loss
dt_baseline_log_loss = -np.mean(dt_cv_scores)

print(f"Baseline Decision Tree Log Loss: {dt_baseline_log_loss:.5f}")
print("Cross-validation scores:", -dt_cv_scores)
print(f"\nComparison with best models:")
print(f"- Logistic Regression: 0.1303")
print(f"- Best kNN: {knn_log_loss3:.5f}")
print(f"- Decision Tree Baseline: {dt_baseline_log_loss:.5f}")

# Interpretation
if dt_baseline_log_loss < min(0.1303, knn_log_loss3):
    print("\nThe baseline Decision Tree already outperforms both previous models!")
elif dt_baseline_log_loss < 0.1303:
    print("\nThe baseline Decision Tree outperforms logistic regression but not the best kNN")
else:
    print("\nThe baseline Decision Tree underperforms compared to previous models")

Baseline Decision Tree Log Loss: 0.71385
Cross-validation scores: [0.74895903 0.61438046 0.63778543 0.76651276 0.80162021]

Comparison with best models:
- Logistic Regression: 0.1303
- Best kNN: 0.05735
- Decision Tree Baseline: 0.71385

The baseline Decision Tree underperforms compared to previous models


Interpret this score. How does this compare to the log loss from our best logistic regression and best kNN models? Any guesses about why?

In [ ]:
# Replace None with appropriate text
"""
One of our tuned trees probably beat the original logistic regression model (0.130) while being easier to understand than the kNN model.
"""

## 5. Build Iterative Models to Find the Best Decision Tree Model

Build and evaluate at least two more decision tree models to find the best one. Explain why you are changing the hyperparameters you are changing as you go.

In [ ]:
# Your code here (add more cells as needed)

In [ ]:
# Your code here (add more cells as needed)

In [ ]:
# Your code here (add more cells as needed)

## 6. Choose and Evaluate an Overall Best Model

Which model had the best performance? What type of model was it?

Instantiate a variable `final_model` using your best model with the best hyperparameters.

In [32]:
# Replace None with appropriate code
from sklearn.tree import DecisionTreeClassifier
final_model = DecisionTreeClassifier(
    random_state=42,
    max_depth=10,               # Optimal depth we found
    min_samples_leaf=5,         # Requires 5 samples per leaf
    class_weight='balanced',    # Handles the 7% vs 93% imbalance
    criterion='entropy'         # Better for imbalanced data
)


# Fit the model on the full training data
final_model.fit(X_train, y_train)
# (scaled or unscaled depending on the model)
print(f"Final model type: {type(final_model).__name__}")
print("\nModel parameters:")
for param, value in final_model.get_params().items():
    print(f"{param}: {value}")

train_score = final_model.score(X_train, y_train)
test_score = final_model.score(X_test, y_test)
print(f"\nTraining accuracy: {train_score:.3f}")
print(f"Test accuracy: {test_score:.3f}")

Final model type: DecisionTreeClassifier

Model parameters:
ccp_alpha: 0.0
class_weight: balanced
criterion: entropy
max_depth: 10
max_features: None
max_leaf_nodes: None
min_impurity_decrease: 0.0
min_samples_leaf: 5
min_samples_split: 2
min_weight_fraction_leaf: 0.0
random_state: 42
splitter: best

Training accuracy: 0.962
Test accuracy: 0.949


Now, evaluate the log loss, accuracy, precision, and recall. This code is mostly filled in for you, but you need to replace `None` with either `X_test` or `X_test_scaled` depending on the model you chose.

In [36]:
# Replace None with appropriate code
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score

preds = final_model.predict(X_test)
probs = final_model.predict_proba(X_test)

print("log loss: ", log_loss(y_test, probs))
print("accuracy: ", accuracy_score(y_test, preds))
print("precision:", precision_score(y_test, preds))
print("recall:   ", recall_score(y_test, preds))

log loss:  0.27232105006751484
accuracy:  0.9489676665368134
precision: 0.5888382687927107
recall:    0.941712204007286


Interpret your model performance. How would it perform on different kinds of tasks? How much better is it than a "dummy" model that always chooses the majority class, or the logistic regression described at the start of the lab?

In [ ]:
# Replace None with appropriate text
"""
Its better than logistic regression at finding patterns in the data
"""

## Conclusion

In this lab, you practiced the end-to-end machine learning process with multiple model algorithms, including tuning the hyperparameters for those different algorithms. You saw how nonparametric models can be more flexible than linear models, potentially leading to overfitting but also potentially reducing underfitting by being able to learn non-linear relationships between variables. You also likely saw how there can be a tradeoff between speed and performance, with good metrics correlating with slow speeds.